In [1]:
import os

from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

In [ ]:
# config file_dir, work_dir 수정 후 사용

# config file 들고오기
name = 'knet_upernet_swinl_AdamW_iter500_aug'
cfg = Config.fromfile(f'./configs/_custom_/models/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k.py') # load config file
# root='../input/mmseg/test/'
root='/opt/ml/input/data/mmseg/test'
epoch = 'best_mIoU_epoch_26'

# dataset config 수정
cfg.work_dir = f'/opt/ml/mmseg/saved/{name}' # set work_dir
cfg.data.test.img_dir = root
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True
cfg.data.samples_per_gpu = 1
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

In [3]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2022-05-12 08:54:09,680 - mmseg - INFO - Loaded 624 images


In [4]:
model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/mmseg/mmseg/models/backbones/swin.py:556: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
/opt/ml/mmseg/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '


load checkpoint from local path: /opt/ml/mmseg/saved/knet_upernet_swinl_AdamW_iter500_aug/best_mIoU_epoch_26.pth


In [ ]:
output = single_gpu_test(model, data_loader)

In [ ]:
# sample_submisson.csv 열기
# submission = pd.read_csv('../output/sample_submission.csv', index_col=None)
# json_dir = os.path.join("../input/data/test.json")
submission = pd.read_csv('/opt/ml/input/code/submission/sample_submission.csv', index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

input_size = 512
output_size = 256
bin_size = input_size // output_size
		
# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    
    temp_mask = []
    predict = predict.reshape(1, 512, 512)
    # resize predict to 256, 256
    # reference : https://stackoverflow.com/questions/48121916/numpy-resize-rescale-image
    mask = predict.reshape((1, output_size, bin_size, output_size, bin_size)).max(4).max(2) 
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()

    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join('/opt/ml/mmseg/submission/', f'{name}_{epoch}_multiTTA.csv'), index=False)